In [ ]:
!pip install torch
!pip install transformers datasets rouge_score
import logging
import os
import re
import sys
from dataclasses import dataclass, field
from typing import Optional

import numpy as np
import pandas as pd
import nltk
import torch
import transformers
from datasets import load_dataset, load_metric, load_from_disk
from rouge_score import rouge_scorer
from tqdm import tqdm
from transformers import (
    AutoConfig,
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    DataCollatorForSeq2Seq,
    HfArgumentParser,
    MBartTokenizer,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    default_data_collator,
    set_seed,
)
from transformers.trainer_utils import is_main_process
from datasets import load_dataset

transformers.logging.set_verbosity_info()

DATA_PATH = "data/"

In [ ]:
import random
import json

In [ ]:
from google.colab import drive
drive.mount("/content/drive")
%cd "/content/drive/My Drive/CS/cs287_project/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/.shortcut-targets-by-id/1U8uJcoCvW4AqtO1k_6Bd44D-dqUdVEXS/cs287_project


In [ ]:
%cd "/content/drive/My Drive/CS/cs287_project/"

/content/drive/.shortcut-targets-by-id/1U8uJcoCvW4AqtO1k_6Bd44D-dqUdVEXS/cs287_project


In [ ]:
!nvidia-smi

Sun Dec 12 03:20:50 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
"""
Load model
"""

MODEL_NAME = "./exp/bart_corrupted_predicate/results/checkpoint-180000"
max_target_length = 128
max_source_length = 1024
padding = False

config = AutoConfig.from_pretrained(
    MODEL_NAME, cache_dir=None, revision="main", use_auth_token=False,
    max_position_embeddings = max_source_length

)

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME, cache_dir=None, use_fast=True, revision="main", use_auth_token=False,
)

model = AutoModelForSeq2SeqLM.from_pretrained(
    MODEL_NAME,
    from_tf=bool(".ckpt" in MODEL_NAME),
    config=None,
    cache_dir=None,
    revision="main",
    use_auth_token=False,
)

# Set decoder_start_token_id
if model.config.decoder_start_token_id is None and isinstance(tokenizer, MBartTokenizer):
    model.config.decoder_start_token_id = tokenizer.lang_code_to_id[data_args.target_lang]
if model.config.decoder_start_token_id is None:
    raise ValueError


loading configuration file ./exp/bart_corrupted_predicate/results/checkpoint-180000/config.json
Model config BartConfig {
  "_name_or_path": "./exp/bart_corrupted_predicate/results/checkpoint-180000",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 12,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 2,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encode

In [ ]:
xsum_test = load_dataset("xsum", split="test")


Using custom data configuration default
Reusing dataset xsum (/root/.cache/huggingface/datasets/xsum/default/1.2.0/32c23220eadddb1149b16ed2e9430a05293768cfffbdfd151058697d4c11f934)


In [ ]:
xsum_train = load_dataset("xsum", split="train")

Downloading:   0%|          | 0.00/2.05k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/954 [00:00<?, ?B/s]

Using custom data configuration default


  0%|          | 0/2 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/255M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.00M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset xsum downloaded and prepared to /root/.cache/huggingface/datasets/xsum/default/1.2.0/32c23220eadddb1149b16ed2e9430a05293768cfffbdfd151058697d4c11f934. Subsequent calls will reuse this data.


In [ ]:
os.chdir('../..')

In [ ]:
model.to('cuda')

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0): BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
        

In [ ]:
os.chdir('../..')


In [ ]:
def get_docs(data):
    documents = {}
    for d in data:
        documents[d['id']] = [d['document'], d['summary']]
    
    return documents

In [ ]:
ls

corenlp/                               corenlp_server-7471222b81f34749.props
corenlp_server-0b0a38ab415e4083.props  corenlp_server-d5f55ab414244b9a.props
corenlp_server-2948a76abe404085.props  data/
corenlp_server-36fe0852703c419f.props  exp/
corenlp_server-6aff972585d14c53.props  ids_to_keep.npy


In [1]:
import pickle

random.seed(2021)


file_path = f"../../../MyDrive/CS/data/test_predicate.pkl"
pickle_off = open(file_path,"rb")
corrupted_dict = pickle.load(pickle_off)
full_documents = []
all_sums = []

ids = random.sample(list(corrupted_dict.keys()), 100)
#corrupted_ids = random.sample(ids, 50)
doc_dict = get_docs(xsum_test)

for id in ids:
  # if id in corrupted_ids:
  #   all_sums.append(corrupted_dict[id])
  # else:
  #   all_sums.append(doc_dict[id][1])
  all_sums.append(corrupted_dict[id])
  full_documents.append(doc_dict[id][0])




NameError: ignored

In [ ]:
def read_text_file(file_path):
    text_file = ""
    with open(file_path, 'rb') as f:
        text_file += str(f.read())
    
    return text_file

In [ ]:
os.chdir('../..')

In [ ]:
random.seed(200)

doc_dict = get_docs(xsum_test)

os.chdir('./data/bart-large-xsum-summaries')

baseline_sums = []
ids = []
files = os.listdir()
sample_files = random.sample(files, 1000)
for file in sample_files:
   if file.endswith(".model_summary"):
    file_path = f"{file}"
    id = file_path.split('.')[0]
    # call read text file function
    summary = read_text_file(file_path)
    baseline_sums.append(summary[2:-1])
    ids.append(id)
os.chdir('../..')

full_documents = []
for i in range(len(ids)):
  full_documents.append(doc_dict[ids[i]][0])


print(full_documents[0])
print(baseline_sums[0])

EDF boss Jean-Bernard Levy joined officials from the UK, France and China at the behind-closed-doors ceremony in London on Thursday.
Earlier this month, the government gave the go-ahead for the plant which will power nearly six million homes.
It will be the UK's first new nuclear plant in a generation.
The meeting was a second attempt at finalising the deal after Prime Minister Theresa May unexpectedly announced in July that she wanted time to scrutinise it.
With her approval now granted and the contracts updated, the formal signing took place at a ceremony attended by Business Secretary Greg Clark and French Foreign Minister Jean-Marc Ayrault.
China's National Energy Administration director, Nur Bekri, and the chairman of Chinese firm CGN, He Yu, also attended.
Mr Clark said that signing the contract was a "crucial moment" and that Hinkley played "an important part in ensuring our future low-carbon energy security".
Mr Ayrault said he was pleased that the UK government had decided to 

In [ ]:
all_sums = baseline_sums

In [ ]:
generated = {}

for i in range(0, len(ids)):
  id = ids[i]
  tokens = tokenizer.encode(text= all_sums[i], text_pair=full_documents[i], return_tensors='pt', max_length=1024, truncation='only_second')
  #tokens = tokenizer.encode(text= full_documents[i], text_pair=all_sums[i], return_tensors='pt', max_length=1024, truncation='only_first')
  outputs = model.generate(tokens.to('cuda'), max_length=50, min_length=5, num_beams=5)
  generated[id] = tokenizer.decode(outputs[0])

In [ ]:
print(generated)

{'30840160': '</s><s>Two suspected jihadists have been killed in an anti-terror operation in eastern Belgium, officials demurbing.</s>', '27838706': '</s><s>Paul Sheerin has left his post as Arbroath manager to come Under-20s coach at Aberdeen.</s>', '27890466': '</s><s>The RSPCA has reseated a 100% success rate for its prosecutions after facing a "growing animal cruelty crisis".</s>', '35893346': '</s><s>A 16-year-old girl who attacked a police officer inside Dundee Sheriff Court after being ejected from the building has been sentenced to be barren for 12 months.</s>', '38934959': '</s><s>About 1,650 staff and students will have to move out of an Oxford University building for up to two years after asbestos was discovered.</s>', '35982080': '</s><s>The warnings about the forthcoming all-out junior doctor strikes have been coming in thusly thick and fast that you could be forgiven for thinking it will be Armageddon.</s>', '39847422': "</s><s>Performers from 42 countries strode down a l

In [ ]:
comparison = {}
number_changed = 0

for i in range(len(ids)):
  new = generated[ids[i]][7:-4]
  old = all_sums[i]
  real = doc_dict[ids[i]][1]
  comparison[ids[i]] = [new, old]
  # if ids[i] in corrupted_ids:
  #   print("CORRUPTED")
  # else:
  #   print("UNCORRUPTED")
  if new != old:
    print(ids[i])
    print("CHANGED")
    number_changed += 1
    print(full_documents[i][:1024])
    print("Entity Model Generated: " + new)
    #print("Predicate Model Generated: " + new)
    print("Corrupted Summary: " + old)
    print("True Summary: " + real)
    print()
    print()


print(number_changed)

36362284
CHANGED
The settlement was reached in a compensation claim against the Northern Health and Social Care Trust.
The child, who cannot be identified, suffered hypoxia due to a delay in her delivery at Antrim Area Hospital in 2007.
It resulted in a series of complex health issues.
These included severe cerebral palsy and complex epilepsy.
Her parents sued the trust, claiming medical negligence.
The case had been due to run as a five-day trial at the High Court in Belfast earlier this month.
However, full liability was admitted a week before the listed hearing date.
The figure of £5.3m covers damages and the costs of past and future care for the child.
Entity Model Generated: A Northern Ireland health trust has agreed to pay £35.3m to the parents of a baby girl who suffered severe health problems after birth.
Corrupted Summary: A Northern Ireland health trust has agreed to pay \xa35.3m to the parents of a baby girl who suffered severe health problems after birth.
True Summary: An e

In [ ]:
with open('../../../MyDrive/CS/data/test_predicate_predicate.pkl', 'wb') as file:
  pickle.dump(comparison, file)


In [ ]:
print(baseline_sums)

['Shares in BT rose sharply on Wednesday morning, after the regulator Ofcom said it would not force the company to spin off its Openreach division.', 'Labour is "trying to get a compromise" on the Brexit bill, the shadow foreign secretary has said.', "Torquay United manager Kevin Nicholson says he will not receive any money from the sale of Kane O'Kane to Bournemouth.", "Andy Murray's season has not gone to plan.", 'More than 1,800 people have signed a petition against plans to move a statue commemorating the Falklands War.', "US President Barack Obama has announced new sanctions against Russia in response to Moscow's alleged interference in the US election.", 'Sale Sharks director of rugby Les Cotton says the club need to rebuild if they are to compete at the top level.', 'British number one Johanna Konta reached the quarter-finals of the Nottingham Open with a straight-set win over Madison Brengle.', 'Former world number one Stuart Bingham says he could return to the World Championsh

In [ ]:
print(doc_dict['39502281'])

['The bird, known officially as LF15, and her partner LM12 bred three chicks in both 2015 and 2016 at the Dunkeld reserve.\nLassie arrived back in Scotland on 23 March, six days after her mate.\nViewers from around the world have been watching the pair\'s progress on the Loch of the Lowes live webcam.\nCharlotte Fleming, Perthshire ranger for the Scottish Wildlife Trust, said: "LF15 has settled down to begin incubating, and we would expect the egg to hatch in about six weeks\' time.\n"It\'s now clear that the pair\'s attempts at breeding have been a success and they are on track to repeat the hat-trick of chicks that were hatched in both 2015 and 2016."', "A female osprey nicknamed Lassie has laid her first egg of the season at the Scottish Wildlife Trust's Loch of the Lowes nature reserve."]


In [ ]:
print(doc_dict['33847589'])

['The sum is on top of more than £1.2bn set aside in previous years\' provisions.\nParent company National Australian Bank (NAB) said between £350m and £500m more would be set aside this year, but added that the final cost remained uncertain.\nThe move was announced in its third quarter results on Monday.\nUp until the end of March, NAB had set aside £806m to compensate customers for the mis-selling of payment protection insurance (PPI). A total of £325m has already been paid out.\nNAB\'s cumulative provisions relating to interest rate hedging products amounted to £431m, about half of which has been paid out.\nThis year\'s extra funding will come from a £1.7bn mis-selling fund, which the bank is being required by UK regulators to make available after the Clydesdale is floated.\nNAB is in the process of demerging and floating Glasgow-based Clydesdale, which includes the Yorkshire Bank.\nIt intends to complete the break-up by the end of this year.\nIn a statement, NAB said "substantial p